<a href="https://colab.research.google.com/github/Maumau-3005/Hydra-fullstack/blob/main/webscrapping_hydra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
df = pd.read_csv("dataset_obesity-2024_2.csv")
list(df.columns)

['Gender',
 'Age',
 'Height',
 'Weight',
 'family_history_with_overweight',
 'FAVC',
 'FCVC',
 'NCP',
 'CAEC',
 'SMOKE',
 'CH2O',
 'SCC',
 'FAF',
 'TUE',
 'CALC',
 'MTRANS',
 'NObeyesdad',
 'TW1']

In [ ]:
obeyesSmoke = df[["NObeyesdad","SMOKE"]].value_counts()
display(obeyesSmoke)

,,count
NObeyesdad,SMOKE,
Normal Weight,no,1117
Obesity,no,950
Normal Weight,yes,22
Obesity,yes,22
